## Demo 001 - Conceitos iniciais



### Demo para mostrar os principios de como criar algumas formas de criar uma tabela e como popular

#### Para separar nossos dados vamos de inicio começar criando um catalogo proprio nosso para esse laboratório

![](https://learn.microsoft.com/pt-br/azure/databricks/_static/images/unity-catalog/object-model.png)

#### Um catalogo é o 3o nivel hierarquico dentro do databricks. 
#### Acima dele fica somente a metastore aonde efetivamente ficam os dados, abaixo dele temos schemas e abixo dos schemas todos os demais objetos

In [ ]:
%sql
-- CREATE CATALOG jorge

Abaixo dele vai ficar o schema, vamos criar um dentro dele para podermos colocar nossos objetos

In [ ]:
%sql
-- CREATE SCHEMA jorge.workshop

Por fim vamos criar uma tabela de cores dentro do schema que criamos

In [ ]:
%sql
CREATE TABLE default.cores  (
  idCor int,
  descCor string
)

Vamos ver o que temos dentro da nossa tabela

In [ ]:
%sql
SELECT * FROM default.cores

Vamos inserir manualmente dados dentro dela

In [ ]:
%sql
INSERT INTO default.cores VALUES ( 1, "Preto") , (2, "Branco") , (3, "Azul")

Vamos ver agora os dados dela

In [ ]:
%sql
SELECT * FROM default.cores

#### Lendo os dados da tabela usando python

In [ ]:
df_cores = spark.sql("SELECT * FROM default.cores")

df_cores.display()


#### Criando nova coluna usando python

In [ ]:
from pyspark.sql.functions import lit

df_cores = df_cores.withColumn("criadorCor", lit("Jorge"))

df_cores.display()


#### Criando tempView para poder fazer `leitura` da tabela criada com python

In [ ]:
df_cores.createOrReplaceTempView("temp_python_cores")

In [ ]:
%sql
SELECT * FROM temp_python_cores


Perceba que é uma nova tabela temporal, a tabela `cores` contonua existindo, mas não estamos alterando ela

In [ ]:
%sql
SELECT * FROM default.cores

#### Criando tempView com SQL

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW temp_sql_cores AS
SELECT * 
FROM temp_python_cores

#### Lendo os dados da tempView de SQL usando python

In [ ]:
df_temp_cores = spark.sql("SELECT * FROM temp_sql_cores")

df_temp_cores.display()

## Lendo Arquivo upload

Primeiro precisamos fazer o upload do arquivo

In [ ]:
caminho_arquivo = "dbfs:/FileStore/shared_uploads/jchambyd@gmail.com/Sales_Data.csv"

# Lê o arquivo CSV e infere o schema automaticamente
dataframe = spark.read.csv(caminho_arquivo, header=True, inferSchema=True)

# Renomeamos a coluna 'Quantity Sold'
dataframe = dataframe.withColumnRenamed("Quantity Sold", "QuantitySold")
dataframe = dataframe.withColumnRenamed("Sub Brand", "SubBrand")

display(dataframe)


Agora salvamos como tabela dentro do databricks


In [ ]:
dataframe.write.mode("overwrite").saveAsTable("SalesData")


Consultamos os dados da tabela

In [ ]:
%sql
SELECT * FROM SalesData